In [ ]:
!pip install torch_geometric
!pip install torchinfo


## Preprocessing
Here, we preprocess raw ID-paired edges into the PyG dataset format.

In [ ]:
import torch
from torch_geometric.data import InMemoryDataset, Data

class CustomGraphDataset(InMemoryDataset):
    def __init__(self, root, filename, transform=None, pre_transform=None):
        self.filename = filename
        super().__init__(root, transform, pre_transform)

        # Allowlist PyG classes
        from torch_geometric.data.data import DataEdgeAttr
        torch.serialization.add_safe_globals([DataEdgeAttr])

        self.data, self.slices = torch.load(self.processed_paths[0], weights_only= False)

    @property
    def raw_file_names(self):
        return [self.filename]  # Dynamically return the provided filename

    @property
    def processed_file_names(self):
        # Generate a unique processed filename based on the input file
        return [f"{self.filename.split('.')[0]}.pt"]

    def download(self):
        # Add download logic here if needed
        pass

    def process(self):
        # Read edges from the specified file
        edges = []
        with open(self.raw_paths[0], 'r') as f:
            for line in f:
                src, tgt = line.strip().split()
                edges.append([int(src), int(tgt)])

        # Create edge_index tensor
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        # Calculate Number of nodes using edge_index
        node_set = set()
        for edge in edges:
            node_set.add(edge[0])
            node_set.add(edge[1])
        num_nodes = len(node_set)
        # Create Data object
        data = Data(edge_index=edge_index, num_nodes=num_nodes)
        data.x = torch.arange(num_nodes, dtype=torch.long)
        # Apply optional pre-processing
        if self.pre_transform is not None:
            data = self.pre_transform(data)

        # Save processed data
        torch.save(self.collate([data]), self.processed_paths[0])

# Usage Example
dataset = CustomGraphDataset(
    root='',
    filename='raw_edges.txt'  # Specify which file to load
)



Data overview

In [ ]:
def print_dataset_info(dataset, dataset_name="Dataset"):
    """Prints basic information about a PyG dataset."""
    data = dataset[0]  # Get the graph object

    print(f"\n=== {dataset_name} Information ===")
    print(f"Number of nodes: {data.num_nodes}")
    print(f"Number of edges: {data.edge_index.size(1)}")
    print(f"Contains node features: {'x' in data}")
    print(f"Contains edge features: {'edge_attr' in data}")
    print(f"Contains graph labels: {'y' in data}")
print_dataset_info(dataset)



## Split dataset
we are splitting the dataset into three distinct subsets: the training graph, validation graph, and testing graph. The purpose of this split is to ensure that our model can learn effectively, validate its performance during training, and evaluate its generalization capabilities on unseen data.
- The testing graph has the largest number of edges due to the masking process applied to the original dataset.
- By carefully selecting edges for each subset, we create a testing environment while still providing sufficient data for training and validation.

In [ ]:
from torch_geometric.transforms import RandomLinkSplit
edge_transform = RandomLinkSplit(num_val=0.2, num_test=0.2,
                                is_undirected=True, add_negative_train_samples=False)
train_data, val_data, test_data = edge_transform(dataset.data)
print(train_data)
print(val_data)
print(test_data)

## GraphSAGE
Below is the implementation for GraphSAGE, please fill in the TODO blanks.

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score
from torch.nn import Embedding
class GraphSAGE(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, hidden_dim, out_dim, num_layers=2):
        super().__init__()
        self.embedding = # TODO
        
        # Create layer sequence
        self.convs = torch.nn.ModuleList()
        in_dim = embedding_dim
        for _ in range(num_layers-1):
            self.convs.append(SAGEConv(in_dim, hidden_dim))
            in_dim = hidden_dim
        self.convs.append(SAGEConv(in_dim, out_dim))

    def forward(self, x, edge_index):
        x = self.embedding(x)
        for conv in self.convs:
            # TODO
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.lin = # TODO

    def forward(self, z, edge_index):
        src = z[edge_index[0]]
        dst = z[edge_index[1]]
        return self.lin(torch.cat([src, dst], dim=1)).squeeze()

## Configure device for faster training

In [5]:
from torchinfo import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cuda
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')# apple

In [6]:
def train(model_graphsage, predictor, optimizer):
    model_graphsage.train()
    predictor.train()
    optimizer.zero_grad()

    # Forward pass to get node embeddings
    z = model_graphsage(train_data.x.to(device), train_data.edge_index.to(device))

    # Positive edges
    pos_edge_index = train_data.edge_label_index.to(device)
    # Negative sampling
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index.to(device),
        num_nodes=train_data.num_nodes,
        num_neg_samples=pos_edge_index.size(1),
    ).to(device)

    # Combine positive and negative edges
    edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=1)
    labels = torch.cat([
        torch.ones(pos_edge_index.size(1)),
        torch.zeros(neg_edge_index.size(1))
    ], dim=0).to(device)

    # Prediction scores
    scores = predictor(z, edge_index)

    # Loss and backwarde
    loss = F.binary_cross_entropy_with_logits(scores, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model_graphsage, predictor, data):
    model_graphsage.eval()
    predictor.eval()
    with torch.no_grad():
        z = model_graphsage(data.x.to(device), data.edge_index.to(device))
        scores = predictor(z, data.edge_label_index.to(device))
        pred = scores.sigmoid()
        loss = F.binary_cross_entropy_with_logits(scores, data.edge_label.to(device))
        auc = roc_auc_score(data.edge_label.cpu().numpy(), pred.cpu().numpy())

        # Calculate additional metrics
        threshold = 0.5
        predicted_labels = (pred >= threshold).int()
        precision = precision_score(data.edge_label.cpu().numpy(), predicted_labels.cpu().numpy())
        recall = recall_score(data.edge_label.cpu().numpy(), predicted_labels.cpu().numpy())
        f1 = f1_score(data.edge_label.cpu().numpy(), predicted_labels.cpu().numpy())

        return loss.item(), auc, precision, recall, f1

In [ ]:
from tqdm import tqdm
embedding_dims = # TODO
num_layers_list = # TODO
lr_list = # TODO
# Move data to device
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)
graphsage_auc_scores = {}
verbose = False
for embedding_dim in embedding_dims:
    for num_layers in num_layers_list:
        for lr in lr_list:
            # Initialize model with current parameters
            model = GraphSAGE(
                num_nodes=train_data.num_nodes,
                embedding_dim=embedding_dim,
                hidden_dim=256,  # Fixed hidden dimension
                out_dim=256,
                num_layers=num_layers  # Modified to accept layer count
            ).to(device)

            predictor = LinkPredictor(in_dim=256).to(device)
            optimizer = torch.optim.Adam(
                list(model.parameters()) + list(predictor.parameters()),
                lr=lr
            )
            # Training loop
            for epoch in tqdm(range(1, 20)):
                loss = train(model, predictor, optimizer)
                val_loss, val_auc, val_precision, val_recall, val_f1 = test(model, predictor, val_data)
                test_loss, test_auc, test_precision, test_recall, test_f1 = test(model, predictor, test_data)
                if verbose:
                    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
                    print(f'Validation - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')
                    print(f'Test - Loss: {test_loss:.4f}, AUC: {test_auc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')
                    print('-' * 80)  # Separator line for better readability
                
            _, test_auc, _, _, test_f1 = test(model, predictor, test_data)
            graphsage_auc_scores[(embedding_dim, num_layers, lr)] = test_auc

In [ ]:
# Plot for GraphSAGE
import matplotlib.pyplot as plt
import numpy as np

fixed_lr = 0.01  # Choose a fixed learning rate for the plot
plt.figure(figsize=(8, 6))

# Extract scores for chosen learning rate
a = np.array([[graphsage_auc_scores[(ed, nl, fixed_lr)] 
                for ed in embedding_dims] 
                for nl in num_layers_list])

# Create heatmap
heatmap = plt.imshow(a, cmap="viridis", interpolation="nearest", vmin=0.5, vmax=1.0)
plt.colorbar(heatmap, label="AUC Score")

# Axis labels and ticks
plt.xticks(ticks=range(len(embedding_dims)), 
           labels=[str(d) for d in embedding_dims])
plt.xlabel("Embedding Dimension")

plt.yticks(ticks=range(len(num_layers_list)), 
           labels=[str(n) for n in num_layers_list])
plt.ylabel("Number of Layers")

plt.title(f"Link Prediction Performance (LR={fixed_lr})")

# Add text annotations for the AUC scores
for i in range(len(num_layers_list)):
    for j in range(len(embedding_dims)):
        plt.text(j, i, f"{a[i, j]:.2f}", ha='center', va='center', color='white')

plt.tight_layout()
plt.savefig(f"graphsage_auc_scores_lr_{fixed_lr}.png")
plt.show()

In [9]:
import torch
from torch_geometric.nn import LGConv

class LightGCN(torch.nn.Module):
    def __init__(self, num_nodes, embedding_dim, num_layers=3):
        super().__init__()
        self.embedding = # TODO
        self.convs = torch.nn.ModuleList([LGConv() for _ in range(num_layers)])
        self.num_layers = num_layers


    def forward(self, x, edge_index):
        x = self.embedding(x)
        out = x / (self.num_layers + 1)  # Initial contribution

        for conv in self.convs:
            # TODO

        return out

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.lin = # TODO

    def forward(self, z, edge_index):
        src = z[edge_index[0]]
        dst = z[edge_index[1]]
        return self.lin(torch.cat([src, dst], dim=1)).squeeze()


In [10]:
# Move data to device
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

In [ ]:
from tqdm import tqdm
embedding_dims = # TODO
num_layers_list = # TODO
lr_list = # TODO
# Move data to device
train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)
lightgcn_auc_scores = {}
verbose = False
for embedding_dim in embedding_dims:
    for num_layers in num_layers_list:
        for lr in lr_list:
            # Initialize model with current parameters
            model = LightGCN(
                num_nodes=train_data.num_nodes,
                embedding_dim=embedding_dim,
                num_layers=num_layers  # Modified to accept layer count
            ).to(device)

            predictor = LinkPredictor(in_dim=embedding_dim).to(device)
            optimizer = torch.optim.Adam(
                list(model.parameters()) + list(predictor.parameters()),
                lr=lr
            )
            # Training loop
            for epoch in tqdm(range(1, 20)):
                loss = train(model, predictor, optimizer)
                val_loss, val_auc, val_precision, val_recall, val_f1 = test(model, predictor, val_data)
                test_loss, test_auc, test_precision, test_recall, test_f1 = test(model, predictor, test_data)
                if verbose:
                    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
                    print(f'Validation - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')
                    print(f'Test - Loss: {test_loss:.4f}, AUC: {test_auc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')
                    print('-' * 80)  # Separator line for better readability
                
            _, test_auc, _, _, test_f1 = test(model, predictor, test_data)
            lightgcn_auc_scores[(embedding_dim, num_layers, lr)] = test_auc

## Plot for LightGCN

In [ ]:
# Plot for LightGCN
import matplotlib.pyplot as plt
import numpy as np

fixed_lr = 0.1  # Choose a fixed learning rate for the plot
plt.figure(figsize=(8, 6))

# Extract scores for chosen learning rate
a = np.array([[lightgcn_auc_scores[(ed, nl, fixed_lr)] 
                for ed in embedding_dims] 
                for nl in num_layers_list])

# Create heatmap
heatmap = plt.imshow(a, cmap="viridis", interpolation="nearest", vmin=0.5, vmax=1.0)
plt.colorbar(heatmap, label="auc Score")

# Axis labels and ticks
plt.xticks(ticks=range(len(embedding_dims)), 
           labels=[str(d) for d in embedding_dims])
plt.xlabel("Embedding Dimension")

plt.yticks(ticks=range(len(num_layers_list)), 
           labels=[str(n) for n in num_layers_list])
plt.ylabel("Number of Layers")

plt.title(f"Link Prediction Performance (LR={fixed_lr})")

# Add text annotations for the AUC scores
for i in range(len(num_layers_list)):
    for j in range(len(embedding_dims)):
        plt.text(j, i, f"{a[i, j]:.2f}", ha='center', va='center', color='white')

plt.tight_layout()
plt.savefig(f"lightgcn_auc_scores_lr_{fixed_lr}.png")
plt.show()

## Export Model Weights For Submission

In [ ]:
# Put your best model configs here
model = GraphSAGE(
                num_nodes=train_data.num_nodes,
                embedding_dim= ,# TODO
                hidden_dim=256,  # Fixed hidden dimension
                out_dim=256,
                num_layers=  # TODO
            ).to(device)

predictor = LinkPredictor(in_dim=256).to(device)
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(predictor.parameters()),
    lr= # TODO
)
# Training loop
for epoch in tqdm(range(1, 20)):
    loss = train(model, predictor, optimizer)
    val_loss, val_auc, val_precision, val_recall, val_f1 = test(model, predictor, val_data)
    test_loss, test_auc, test_precision, test_recall, test_f1 = test(model, predictor, test_data)
    if verbose:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Validation - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')
        print(f'Test - Loss: {test_loss:.4f}, AUC: {test_auc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')
        print('-' * 80)  # Separator line for better readability
    
_, test_auc, _, _, _ = test(model, predictor, test_data)
print(f"GraphSAGE Test AUC: {test_auc:.4f}")
torch.save(model.state_dict(), "best_graphsage_model.pth")


model = LightGCN(
                num_nodes=train_data.num_nodes,
                embedding_dim=, # TODO
                num_layers=  # TODO
            ).to(device)

predictor = LinkPredictor(in_dim=embedding_dim).to(device)
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(predictor.parameters()),
    lr= # TODO
)
# Training loop
for epoch in tqdm(range(1, 20)):
    loss = train(model, predictor, optimizer)
    val_loss, val_auc, val_precision, val_recall, val_f1 = test(model, predictor, val_data)
    test_loss, test_auc, test_precision, test_recall, test_f1 = test(model, predictor, test_data)
    if verbose:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        print(f'Validation - Loss: {val_loss:.4f}, AUC: {val_auc:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')
        print(f'Test - Loss: {test_loss:.4f}, AUC: {test_auc:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}')
        print('-' * 80)  # Separator line for better readability
    
_, test_auc, _, _, test_f1 = test(model, predictor, test_data)
print(f"LightGCN Test AUC: {test_auc:.4f}")
torch.save(model.state_dict(), "best_lightgcn_model.pth")
